In [82]:
#imports
import pandas as pd
import requests
import json
import streamlit as st
import numpy as np
import yfinance as yf
from datetime import date
import streamlit as st
from pandas.tseries.offsets import DateOffset
from sklearn.preprocessing import StandardScaler
from sklearn import svm
from sklearn.metrics import classification_report

In [83]:
#ticker_input = st.text_input("Enter the ticker for your stock of choice")

In [84]:
#import stock close price
today = date.today()
starter_ticker = yf.Ticker("^GSPC")
asset_df = starter_ticker.history(period="1d", start="2000-01-01", end=today).drop(columns=["Dividends","Stock Splits","Open", "High","Low","Volume"])
asset_df.index.names = ["timestamp"]
asset_df.tail()

,Close
timestamp,
2023-07-13 00:00:00-04:00,4510.040039
2023-07-14 00:00:00-04:00,4505.419922
2023-07-17 00:00:00-04:00,4522.790039
2023-07-18 00:00:00-04:00,4554.979980
2023-07-19 00:00:00-04:00,4565.720215


In [85]:
#add Actual Returns column
asset_df["Actual Returns"] = asset_df["Close"].pct_change()

In [86]:
#create columns for SMA_fast and SMA_slow
short_window = 4
long_window = 100
asset_df["SMA_Fast"] = asset_df['Close'].rolling(window=short_window).mean()
asset_df["SMA_Slow"] = asset_df['Close'].rolling(window=long_window).mean()

In [87]:
#create signal column
asset_df['Signal'] = 0.0
asset_df.loc[(asset_df['Actual Returns'] >= 0), 'Signal'] = 1
asset_df.loc[(asset_df['Actual Returns'] < 0), 'Signal'] = -1

In [88]:
#create feature and target sets
X = asset_df[['SMA_Fast', 'SMA_Slow']].shift().dropna()
y = asset_df['Signal']

In [89]:
#create X and y train DataFrames
training_begin = X.index.min()
training_end = X.index.min() + DateOffset(years = 18)
X_train = X.loc[training_begin:training_end]
y_train = y.loc[training_begin:training_end]

In [90]:
#create test DataFrames
X_test = X.loc[training_end+DateOffset(hours=1):]
y_test = y.loc[training_end+DateOffset(hours=1):]

In [91]:
#scale the features
scaler = StandardScaler()
X_scaler = scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [92]:
#use SCV model
svm_model = svm.SVC()
svm_model = svm_model.fit(X_train_scaled, y_train)
svm_pred = svm_model.predict(X_test_scaled)
svm_pred[:10]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [94]:
#review class report from model predictions
svm_testing_report = classification_report(y_test, svm_pred)
print(svm_testing_report)

              precision    recall  f1-score   support

        -1.0       0.00      0.00      0.00       599
         1.0       0.54      1.00      0.70       695

    accuracy                           0.54      1294
   macro avg       0.27      0.50      0.35      1294
weighted avg       0.29      0.54      0.38      1294



/Users/nahinhayat/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/nahinhayat/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/nahinhayat/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
#if st.button("Display Close Price"):
    st.write(asset_close__df.tail())